In [1]:
!pip install opencv-python

In [2]:
!pip install mediapipe

In [3]:
!pip install numpy

In [4]:
!pip install pyttsx3

In [ ]:

import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose 

In [ ]:

engine = pyttsx3.init()
def text2speech(text_file):
    pose=text_file
    f=open(pose,'r')
    text=f.read()
    f.close()
    engine.setProperty('rate',150)
    engine.say(text)
    engine.runAndWait()

In [ ]:

def calculate_angle(a,b,c):
    a = np.array(a) 
    b = np.array(b)
    c = np.array(c) 
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
def provide_feedback(label, angles):
    """
    Provide detailed feedback based on the detected pose and angles.
    Includes suggestions for improvement and comparisons to ideal angles.
    """
    feedback = "" 
    
    
    if label == "T Pose":
        
        ideal_angles = {
            'left_elbow': (160, 180), 
            'right_elbow': (160, 180),
            'left_shoulder': (80, 100),
            'right_shoulder': (80, 100),
            'left_knee': (160, 180),
            'right_knee': (160, 180)
        }

       
        if not (ideal_angles['left_elbow'][0] < angles['left_elbow'] < ideal_angles['left_elbow'][1]):
            feedback += "Left elbow angle is too extreme or too bent. Aim for a range between 160° and 180°. "
        if not (ideal_angles['right_elbow'][0] < angles['right_elbow'] < ideal_angles['right_elbow'][1]):
            feedback += "Right elbow angle is too extreme or too bent. Aim for a range between 160° and 180°. "
        if not (ideal_angles['left_shoulder'][0] < angles['left_shoulder'] < ideal_angles['left_shoulder'][1]):
            feedback += "Left shoulder angle is off. Try to align it between 80° and 100°. "
        if not (ideal_angles['right_shoulder'][0] < angles['right_shoulder'] < ideal_angles['right_shoulder'][1]):
            feedback += "Right shoulder angle is off. Try to align it between 80° and 100°. "
        if not (ideal_angles['left_knee'][0] < angles['left_knee'] < ideal_angles['left_knee'][1]):
            feedback += "Left knee angle should be between 160° and 180°. Adjust your posture. "
        if not (ideal_angles['right_knee'][0] < angles['right_knee'] < ideal_angles['right_knee'][1]):
            feedback += "Right knee angle should be between 160° and 180°. Adjust your posture. "

    
    elif label == "Tree Pose":
        
        ideal_angles_tree = {
            'left_knee': (160, 180),
            'right_knee': (160, 180),
            'left_hip': (75, 95),
            'right_hip': (75, 95)
        }
        
        
        if not (ideal_angles_tree['left_knee'][0] < angles['left_knee'] < ideal_angles_tree['left_knee'][1]):
            feedback += "In Tree Pose, left knee angle should be between 160° and 180°. "
        if not (ideal_angles_tree['right_knee'][0] < angles['right_knee'] < ideal_angles_tree['right_knee'][1]):
            feedback += "In Tree Pose, right knee angle should be between 160° and 180°. "
        if not (ideal_angles_tree['left_hip'][0] < angles['left_hip'] < ideal_angles_tree['left_hip'][1]):
            feedback += "Left hip angle in Tree Pose should be between 75° and 95°. "
        if not (ideal_angles_tree['right_hip'][0] < angles['right_hip'] < ideal_angles_tree['right_hip'][1]):
            feedback += "Right hip angle in Tree Pose should be between 75° and 95°. "

    
    elif label == "Warrior II Pose":
        
        ideal_angles_warrior = {
            'left_knee': (160, 180),
            'right_knee': (160, 180),
            'left_shoulder': (70, 90),
            'right_shoulder': (70, 90),
            'left_hip': (90, 100),
            'right_hip': (90, 100)
        }
        
       
        if not (ideal_angles_warrior['left_knee'][0] < angles['left_knee'] < ideal_angles_warrior['left_knee'][1]):
            feedback += "In Warrior II Pose, left knee angle should be between 160° and 180°. "
        if not (ideal_angles_warrior['right_knee'][0] < angles['right_knee'] < ideal_angles_warrior['right_knee'][1]):
            feedback += "In Warrior II Pose, right knee angle should be between 160° and 180°. "
        if not (ideal_angles_warrior['left_shoulder'][0] < angles['left_shoulder'] < ideal_angles_warrior['left_shoulder'][1]):
            feedback += "In Warrior II Pose, left shoulder angle should be between 70° and 90°. "
        if not (ideal_angles_warrior['right_shoulder'][0] < angles['right_shoulder'] < ideal_angles_warrior['right_shoulder'][1]):
            feedback += "In Warrior II Pose, right shoulder angle should be between 70° and 90°. "
        if not (ideal_angles_warrior['left_hip'][0] < angles['left_hip'] < ideal_angles_warrior['left_hip'][1]):
            feedback += "In Warrior II Pose, left hip angle should be between 90° and 100°. "
        if not (ideal_angles_warrior['right_hip'][0] < angles['right_hip'] < ideal_angles_warrior['right_hip'][1]):
            feedback += "In Warrior II Pose, right hip angle should be between 90° and 100°. "
    
    
    else:
        feedback += "Pose not recognized. Please try again."

   
    if not feedback:
        feedback = "Great job! Your pose is correct."

    return feedback


In [ ]:

import cv2
import mediapipe as mp
import numpy as np
import pyttsx3


mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)


label = "Unknown Pose"


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        
        results = pose.process(image)
    
       
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
            right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
            left_knee_angle = calculate_angle(left_ankle, left_knee, left_hip)
            right_knee_angle = calculate_angle(right_ankle, right_knee, right_hip)
           
           
            angles = {
                "left_elbow": left_elbow_angle,
                "right_elbow": right_elbow_angle,
                "left_shoulder": left_shoulder_angle,
                "right_shoulder": right_shoulder_angle,
                "left_hip": left_hip_angle,
                "right_hip": right_hip_angle,
                "left_knee": left_knee_angle,
                "right_knee": right_knee_angle
            }

           
            if left_elbow_angle > 165 and left_elbow_angle < 195 and right_elbow_angle > 165 and right_elbow_angle < 195:
                if left_shoulder_angle > 80 and left_shoulder_angle < 110 and right_shoulder_angle > 80 and right_shoulder_angle < 110:
                    
                    if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:
                        if left_knee_angle > 90 and left_knee_angle < 120 or right_knee_angle > 90 and right_knee_angle < 120:
                            label = "Warrior II Pose"
                            feedback = provide_feedback(label, angles)
                            text2speech(feedback)
                   
                    if left_knee_angle > 160 and left_knee_angle < 195 and right_knee_angle > 160 and right_knee_angle < 195:
                        label = 'T Pose'
                        feedback = provide_feedback(label, angles)
                        text2speech(feedback)
            
            if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:
                if left_knee_angle > 315 and left_knee_angle < 335 or right_knee_angle > 25 and right_knee_angle < 45:
                    label = 'Tree Pose'
                    feedback = provide_feedback(label, angles)
                    text2speech(feedback)
                    
            else:
                label = 'Unknown Pose'
        
        except Exception as e:
            print(e)
            pass
        
       
        cv2.putText(image, label, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1, cv2.LINE_AA)
        
       
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
                
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in

[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left h

'NoneType' object has no attribute 'landmark'
[Errno 2] No such file or directory: 'Great job! Your pose is correct.'
[Errno 2] No such file or directory: 'Great job! Your pose is correct.'
[Errno 2] No such file or directory: 'Great job! Your pose is correct.'
[Errno 2] No such file or directory: 'Great job! Your pose is correct.'
[Errno 2] No such file or directory: 'Right shoulder angle is off. Try to align it between 80° and 100°. '
[Errno 2] No such file or directory: 'Great job! Your pose is correct.'
[Errno 2] No such file or directory: 'Great job! Your pose is correct.'
[Errno 2] No such file or directory: 'Great job! Your pose is correct.'
[Errno 2] No such file or directory: 'Great job! Your pose is correct.'
[Errno 2] No such file or directory: 'Great job! Your pose is correct.'
[Errno 2] No such file or directory: 'Right shoulder angle is off. Try to align it between 80° and 100°. '
[Errno 2] No such file or directory: 'Right shoulder angle is off. Try to align it between 8

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute

[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left h

[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left hip angle in Tree Pose should be between 75° and 95°. Right hip angle in Tree Pose should be between 75° and 95°. '
[Errno 2] No such file or directory: 'In Tree Pose, right knee angle should be between 160° and 180°. Left h